# TrustyAI Evaluation Demo

This notebook demonstrates how to use the TrustyAI SDK's evaluation functionality to evaluate language models using the LM Evaluation Harness both locally and on Kubernetes.

## Prerequisites

Make sure you have installed TrustyAI with evaluation support:

```bash
pip install .[eval]
```

Or for all features:

```bash
pip install .[all]
```


## 1. Basic Setup and Imports

First, let's import the necessary modules and check what evaluation providers are available.


In [1]:
from trustyai import Providers
from trustyai.core import DeploymentMode
from trustyai.core.eval import EvaluationProviderConfig

In [2]:
print("=== TrustyAI Providers Class ===")
print("\nAvailable provider types:")
for provider_type in dir(Providers):
    if not provider_type.startswith('_'):
        print(f"  - {provider_type}")

print(f"\nAvailable evaluation providers:")
for provider_name in dir(Providers.eval):
    if not provider_name.startswith('_'):
        print(f"  - {provider_name}")

print(f"\nWe'll use: Providers.eval.LMEvalProvider")
print("The deployment mode in the config will determine whether it runs locally or on Kubernetes.")

=== TrustyAI Providers Class ===

Available provider types:
  - bias_detection
  - eval
  - evaluation
  - explainability

Available evaluation providers:
  - LMEvalProvider
  - Lm-evalProvider

We'll use: Providers.eval.LMEvalProvider
The deployment mode in the config will determine whether it runs locally or on Kubernetes.


## 2. Initialise the Evaluation Provider

Now let's create and initialise the evaluation provider using the new organised Providers class.


In [3]:
# Create the evaluation provider using the new Providers class
provider = Providers.eval.LMEvalProvider()

# Initialise the provider (this will check if lm-eval is available)
try:
    provider.initialize()
    print("\n✓ Evaluation provider initialised successfully!")
    print(f"Provider class: {provider.__class__.__name__}")
    print(f"Provider type: {provider.get_provider_type()}")
    print(f"Supported deployment modes: {[mode.value for mode in provider.supported_deployment_modes]}")
except ImportError as e:
    print(f"\n✗ Error initialising provider: {e}")
    print("Please install evaluation dependencies: pip install .[eval]")


/home/rui/Sync/code/rh/trusty/trustyai-sdk/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



✓ Evaluation provider initialised successfully!
Provider class: LMEvalProvider
Provider type: eval
Supported deployment modes: ['local', 'kubernetes']


## 4. Basic Evaluation Example

Let's run a basic evaluation using a small model and a simple task. We'll use google/flan-t5-base (a small model) and the arc_easy task for demonstration.


In [4]:
# Create evaluation configuration
config = EvaluationProviderConfig(
    evaluation_name="arc_easy",
    model="google/flan-t5-base",  # Small model for quick evaluation
    tasks=["arc_easy"],  # Common sense reasoning task
    limit=5,  # Limit to 5 examples for quick demonstration
    metrics=["acc", "acc_norm"],  # Accuracy metrics
    device="cpu",  # Use CPU to avoid GPU requirements
    deployment_mode=DeploymentMode.LOCAL,
    batch_size=1,  # Small batch size for stability
    num_fewshot=0  # Zero-shot evaluation
)

print("Configuration created:")
print(f"  Model: {config.model}")
print(f"  Tasks: {config.tasks}")
print(f"  Metrics: {config.metrics}")
print(f"  Device: {config.device}")
print(f"  Limit: {config.limit} examples")
print(f"  Batch size: {config.get_param('batch_size')}")
print(f"  Few-shot examples: {config.get_param('num_fewshot')}")

Configuration created:
  Model: google/flan-t5-base
  Tasks: ['arc_easy']
  Metrics: ['acc', 'acc_norm']
  Device: cpu
  Limit: 5 examples
  Batch size: 1
  Few-shot examples: 0


In [5]:
# Run the evaluation
print("Running evaluation...")
try:
    results = provider.evaluate(config)
    print("\n✓ Evaluation completed successfully!")
except Exception as e:
    print(f"\n✗ Evaluation failed: {e}")
    results = None

2025-06-17:21:10:47,459 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'


Running evaluation...
[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-17:21:10:47,827 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-17:21:10:53,121 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-17:21:10:53,122 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-17:21:10:55,849 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of arc_easy from None to 0
2025-06-17:21:10:55,850 INFO     [lm_eval.api.task:420] Building contexts for arc_easy on rank 0...
100%|██████████| 5/5 [00:00<00:00, 2473.06it/s]
2025-06-17:21:10:55,853 INFO     [lm_eval.evaluator:525] Running loglikelihood requests
Running loglikelihood requests: 100%|██████████| 20/20 [00:00<00:00, 21.85it/s]



✓ Evaluation completed successfully!


## 5. Local vs Kubernetes Comparison

Compare the same configuration running locally vs on Kubernetes. This demonstrates how the same provider handles both deployment modes.


In [6]:
# Shared configuration for both deployments
shared_config = {
    "evaluation_name": "comparison_demo",
    "model": "google/flan-t5-base",
    "tasks": ["arc_easy"],
    "limit": 3,  # Small limit for quick comparison
    "metrics": ["acc", "acc_norm"],
    "batch_size": 1,
    "num_fewshot": 0
}

# Configuration for LOCAL deployment
local_config = EvaluationProviderConfig(
    **shared_config,
    deployment_mode=DeploymentMode.LOCAL,
    device="cpu"
)

# Configuration for KUBERNETES deployment
kubernetes_config = EvaluationProviderConfig(
    **shared_config,
    deployment_mode=DeploymentMode.KUBERNETES,
    namespace="test",
    deploy=True,
    wait_for_completion=True,
    timeout=300
)

print("🔄 RUNNING LOCAL vs KUBERNETES COMPARISON")
print("=" * 50)


🔄 RUNNING LOCAL vs KUBERNETES COMPARISON


In [7]:
# Run local evaluation
print(f"📍 LOCAL Configuration:")
print(f"  Deployment mode: {local_config.deployment_mode.value}")
print(f"  Device: {local_config.device}")

print("\nRunning local evaluation...")
local_results = None
try:
    local_results = provider.evaluate(local_config)
    print("✓ Local evaluation completed!")
except Exception as e:
    print(f"✗ Local evaluation failed: {e}")


2025-06-17:21:11:21,621 INFO     [lm_eval.models.huggingface:136] Using device 'cpu'


📍 LOCAL Configuration:
  Deployment mode: local
  Device: cpu

Running local evaluation...
[DEBUG - _parse_args_to_config] Args=1: has namespace? False
Using device: cpu for model evaluation


2025-06-17:21:11:21,891 INFO     [lm_eval.models.huggingface:376] Model parallel was set to False, max memory was not set, and device map was set to {'': 'cpu'}
2025-06-17:21:11:27,198 INFO     [lm_eval.evaluator:177] Setting random seed to 0 | Setting numpy seed to 1234 | Setting torch manual seed to 1234 | Setting fewshot manual seed to 1234
2025-06-17:21:11:27,198 INFO     [lm_eval.evaluator:230] Using pre-initialized model
2025-06-17:21:11:29,205 WARNING  [lm_eval.evaluator:283] Overwriting default num_fewshot of arc_easy from None to 0
2025-06-17:21:11:29,206 INFO     [lm_eval.api.task:420] Building contexts for arc_easy on rank 0...
100%|██████████| 3/3 [00:00<00:00, 2394.01it/s]
2025-06-17:21:11:29,208 INFO     [lm_eval.evaluator:525] Running loglikelihood requests
Running loglikelihood requests: 100%|██████████| 12/12 [00:00<00:00, 20.40it/s]


✓ Local evaluation completed!


In [8]:
# Helper function for Kubernetes evaluation
async def run_kubernetes_evaluation(provider, config):
    """Run Kubernetes evaluation asynchronously."""
    try:
        results = await provider.evaluate(config)
        print("✓ Kubernetes evaluation completed!")
        return results
    except Exception as e:
        print(f"✗ Kubernetes evaluation failed: {e}")
        return {"status": "failed", "error": str(e)}

# Run Kubernetes evaluation (async)
print(f"🚀 KUBERNETES Configuration:")
print(f"  Deployment mode: {kubernetes_config.deployment_mode.value}")
print(f"  Namespace: {kubernetes_config.get_param('namespace')}")
print(f"  Deploy: {kubernetes_config.get_param('deploy')}")
print(f"  Wait for completion: {kubernetes_config.get_param('wait_for_completion')}")

print("\nRunning Kubernetes evaluation...")
kubernetes_results = await run_kubernetes_evaluation(provider, kubernetes_config)


🚀 KUBERNETES Configuration:
  Deployment mode: kubernetes
  Namespace: test
  Deploy: True
  Wait for completion: True

Running Kubernetes evaluation...
[DEBUG - _parse_args_to_config] Args=1: has namespace? True
[DEBUG - _parse_args_to_config] Namespace value: test
[DEBUG - _evaluate_kubernetes_async] Config keys: ['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'additional_params', 'deployment_mode', 'device', 'evaluation_name', 'get_param', 'limit', 'metrics', 'model', 'tasks']
[DEBUG - _evaluate_kubernetes_async] Config namespace: test
[DEBUG] Using namespace for CR: test
[DEBUG] Setting limit in config as string: 3
[DEBUG] Setting namespace in LMEvalJob resource: test
[DEBUG

In [9]:
# Compare results
print("🔍 RESULTS COMPARISON")
print("=" * 30)

print("\n📊 LOCAL RESULTS:")
print("-" * 20)
if local_results and "results" in local_results:
    for task_name, task_results in local_results["results"].items():
        print(f"✅ Task: {task_name}")
        for metric, value in task_results.items():
            if isinstance(value, (int, float)):
                print(f"   {metric}: {value:.4f}")
else:
    print("❌ No local results available")

print("\n🚀 KUBERNETES RESULTS:")
print("-" * 20)  
if kubernetes_results and "results" in kubernetes_results:
    for task_name, task_results in kubernetes_results["results"].items():
        print(f"✅ Task: {task_name}")
        for metric, value in task_results.items():
            if isinstance(value, (int, float)):
                print(f"   {metric}: {value:.4f}")
else:
    print("❌ No Kubernetes results available")
    if kubernetes_results:
        print(f"Status: {kubernetes_results.get('status', 'unknown')}")
        print(f"Message: {kubernetes_results.get('message', 'no message')}")

print("\n🎉 Demo completed!")


🔍 RESULTS COMPARISON

📊 LOCAL RESULTS:
--------------------
✅ Task: arc_easy
   acc,none: 0.6667
   acc_stderr,none: 0.3333
   acc_norm,none: 0.6667
   acc_norm_stderr,none: 0.3333

🚀 KUBERNETES RESULTS:
--------------------
✅ Task: arc_easy
   acc,none: 0.6667
   acc_stderr,none: 0.3333
   acc_norm,none: 0.6667
   acc_norm_stderr,none: 0.3333

🎉 Demo completed!
